In [9]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
# Define constants
L = np.pi * np.sqrt(5)
a = 2 / 3
tmax = 30
x = np.linspace(0, L, 101)
t = np.linspace(0, tmax, 200)

In [11]:
def phi(x):
    """Initial condition phi(x).
    """
    y = np.zeros_like(x)
    y[(1 < x) & (x < 3)] = np.sin(np.pi * x[(1 < x) & (x < 3)]) ** 3
    return y

In [12]:
def psi(x):
    """Initial velocity psi(x).
    """
    return np.zeros_like(x)

In [13]:
def fourier_u(x, t):
    """Fourier solution for u(x, t)
    """
    y = np.zeros_like(x)
    for k in range(1, 101):
        Xk = np.sin(k * np.pi * x / L)
        Ak = (2 / L) * np.trapezoid(phi(x) * Xk, x)
        Bk = (2 / (a * k * np.pi)) * np.trapezoid(psi(x) * Xk, x)
        Tk = Ak * np.cos(a * k * np.pi * t / L) + Bk * np.sin(a * k * np.pi * t / L)
        y += Tk * Xk
    return y

In [14]:
# Create frames for animation
frames = []
for time in t:
    y = fourier_u(x, time)
    frames.append(
        go.Frame(
            data=[
                go.Scatter(x=x, y=y, mode="lines", line=dict(color="red", width=2)),
                go.Scatter(
                    x=[0], y=[0], mode="markers", marker=dict(color="black", size=10)
                ),
                go.Scatter(
                    x=[L], y=[0], mode="markers", marker=dict(color="black", size=10)
                ),
            ],
            name=f"t={time:.2f}",
        )
    )

In [15]:
# Create the initial figure
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=fourier_u(x, 0),
            mode="lines",
            line=dict(color="red", width=2),
            showlegend=False,
        ),
        go.Scatter(
            x=[0],
            y=[0],
            mode="markers",
            marker=dict(color="black", size=10),
            showlegend=False,
        ),
        go.Scatter(
            x=[L],
            y=[0],
            mode="markers",
            marker=dict(color="black", size=10),
            showlegend=False,
        ),
    ],
    frames=frames,
    layout=go.Layout(
        xaxis=dict(range=[0, L], title="x"),
        yaxis=dict(range=[-1, 1], title="u(x, t)"),
        # width=1200,  # Adjust width as needed
        autosize=True,
        # margin=dict(l=0, r=0, t=50, b=50),
        # title="String Vibration", 
        updatemenus=[
            {
                "type": "buttons",
                "showactive": True,
                "x": 0.1,  # Position from left
                "y": -0.02,  # Position from bottom
                "xanchor": "left",
                "yanchor": "top",
                "pad": {"r": 10, "t": 10},
                "direction": "right",  # Horizontal button layout
                "buttons": [
                    {
                        "label": "Play",
                        "method": "animate",
                        "args": [
                            None,
                            {
                                "frame": {"duration": 30, "redraw": True},
                                "fromcurrent": True,
                            },
                        ],
                    },
                    {
                        "label": "Pause",
                        "method": "animate",
                        "args": [
                            [None],
                            {
                                "frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 0},
                            },
                        ],
                    },
                ],
            }
        ],
        sliders=[
            {
                "currentvalue": {"prefix": "t = "},
                "steps": [
                    {
                        "args": [
                            [f.name],
                            {
                                "frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                            },
                        ],
                        "label": str(round(time, 2)),
                        "method": "animate",
                    }
                    for time, f in zip(t, frames)
                ],
            }
        ],
    ),
)

config = {
    'displayModeBar': True,
    'displaylogo': False,
    # 'modeBarButtonsToAdd': ['drawline', 'drawopenpath', 'eraseshape'],
    # 'responsive': True
}
fig.show(config=config)

In [16]:
fig.write_html("fixed_string_plot.html")